In [56]:
import numpy as np
import pandas as pd 
#import matplotlib.pyplot as plt
#import seaborn as sns
pd.set_option('max_columns', 50)
pd.set_option("max_rows", None)
####model libraries ####################
from sklearn.linear_model import Ridge, Lasso,SGDRegressor,LinearRegression
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import ExtraTreeRegressor,DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.neural_network import MLPRegressor
#########################################


from sklearn.metrics import mean_squared_error, mean_absolute_error,make_scorer,r2_score,mean_absolute_percentage_error,explained_variance_score
from sklearn.preprocessing import MinMaxScaler

#### visulization #####
import plotly.graph_objects as go



In [2]:
#df = pd.ExcelFile('C:/Users/halil/Desktop/soalr_data.xlsx').parse('sheet 1')
weather_raw = pd.read_excel('C:/Users/halil/Desktop/soalr_data.xlsx',sheet_name="weather")
guneko_raw = pd.read_excel('C:/Users/halil/Desktop/soalr_data.xlsx',sheet_name="1000255-GUNEKO")

In [5]:
guneko_production = guneko_raw[["Date","Production"]]
guneko_gti = guneko_raw[["Date.1","GTI"]]


weather_guneko = weather_raw.loc[weather_raw.name ==1000255]
ali = pd.merge(guneko_production,weather_guneko,left_on="Date",right_on="date")
dataset = ali.drop(["name","date","lat","lon"],axis=1)

In [6]:
conversion = dataset["Date"].astype(str)

dataset["year"] = conversion.apply(lambda x:int(x[:4]))
dataset["month"] = conversion.apply(lambda x:int(x[5:7]))
dataset["day"] = conversion.apply(lambda x:int(x[8:10]))
dataset["hour"] = conversion.apply(lambda x:int(x[11:13]))
onehot_encoded = dataset.drop(["Date"],axis=1)

In [ ]:

def metrics(y_train1,y_train_pred1,y_test1,y_test_pred1):
    print("for train data, mean squared error is",mean_squared_error(y_train1, y_train_pred1))
    print("for train data, mean absolute error is:",mean_absolute_error(y_train1, y_train_pred1))
    print("for train data, mean absolute percentage error is",mean_absolute_percentage_error(y_train1, y_train_pred1),"\n")

    print("for test data, mean squared error is",mean_squared_error(y_test1, y_test_pred1))
    print("for test data, mean absolute error is:",mean_absolute_error(y_test1, y_test_pred1))
    print("for test data, mean absolute percentage error is",mean_absolute_percentage_error(y_test1, y_test_pred1),"\n")


In [36]:
def calculate_ridge(x_train,x_test,y_train,y_test):
    model = Ridge(alpha=1) 
    model.fit(x_train,y_train)
    train_prediction = model.predict(x_train)
    test_prediction = model.predict(x_test)

    return mean_squared_error(y_train, train_prediction), mean_squared_error(y_test, test_prediction)



def calculate_lasso(x_train,x_test,y_train,y_test):
    model = Lasso(alpha=1) 
    model.fit(x_train,y_train)
    train_prediction = model.predict(x_train)
    test_prediction = model.predict(x_test)

    return mean_squared_error(y_train, train_prediction), mean_squared_error(y_test, test_prediction)




In [46]:
def calculate_linear_reg(x_train,x_test,y_train,y_test):
    
    model = LinearRegression()
    model.fit(x_train, y_train)
    train_prediction = model.predict(x_train)
    test_prediction = model.predict(x_test)
    return mean_squared_error(y_train, train_prediction), mean_squared_error(y_test, test_prediction)

def calculate_xgb(x_train,x_test,y_train,y_test):
    model = XGBRegressor()
    model.fit(x_train, y_train)
    train_prediction = model.predict(x_train)
    test_prediction = model.predict(x_test)
    return mean_squared_error(y_train, train_prediction), mean_squared_error(y_test, test_prediction)

def calculate_kneighbors(x_train,x_test,y_train,y_test):
    model = KNeighborsRegressor()
    model.fit(x_train, y_train)
    train_prediction = model.predict(x_train)
    test_prediction = model.predict(x_test)
    return mean_squared_error(y_train, train_prediction), mean_squared_error(y_test, test_prediction)

def calculate_decisiontree(x_train,x_test,y_train,y_test):
    model = DecisionTreeRegressor()
    model.fit(x_train, y_train)
    train_prediction = model.predict(x_train)
    test_prediction = model.predict(x_test)
    return mean_squared_error(y_train, train_prediction), mean_squared_error(y_test, test_prediction)

def calculate_randomforest(x_train,x_test,y_train,y_test):
    model = RandomForestRegressor()
    model.fit(x_train, y_train)
    train_prediction = model.predict(x_train)
    test_prediction = model.predict(x_test)
    return mean_squared_error(y_train, train_prediction), mean_squared_error(y_test, test_prediction)



In [9]:
#normalization in range (-1,1)
def normalize(df,column_list):
    for col in column_list:
        feature_range = (0,1)
        min_max_scaler = MinMaxScaler(feature_range=feature_range)

        df[col] = min_max_scaler.fit_transform(df[col].values.reshape(-1,1))

    return df

In [ ]:
def make_data(dataframe):
    train = dataframe[:len(dataframe)- 24*5]
    test = dataframe[len(dataframe)- 24*5:]

    x_train = train.copy()
    x_test = test.copy()

    y_train = x_train.pop('Production')
    y_test = x_test.pop('Production')

    return x_train, x_test, y_train, y_test 

In [69]:
for i in models:
    print(str(i)[19:])

<function calculate_ridge at 0x000002A9822F2C80>
<function calculate_lasso at 0x000002A9824862F0>
<function calculate_decisiontree at 0x000002A983202488>
<function calculate_kneighbors at 0x000002A9832022F0>
<function calculate_linear_reg at 0x000002A9832027B8>
<function calculate_randomforest at 0x000002A983202510>
<function calculate_xgb at 0x000002A983202400>


In [10]:
def get_year(whole_data,target_year):

    whole_data = whole_data.loc[whole_data.year== target_year]
    whole_data = whole_data.drop(["year"],axis=1)
    whole_data = whole_data.reset_index(drop=True)

    return whole_data

In [12]:
def get_months(target_data):
    months = []

    for i in range(1,13):
        month_data = target_data.loc[target_data["month"]==i].copy()
        aw1 = month_data.reset_index(drop=True)
        aw1 = aw1.drop(["month"],axis=1)
        months.append(aw1)

    return months

In [57]:
normalized = normalize(onehot_encoded,onehot_encoded.columns[0:11])
data_2021 = get_year(normalized,2021)
data_months = get_months(data_2021)


In [14]:
#x_train, x_test, y_train, y_test = make_data(data_months[0])

In [124]:
def calculate_errors(month_data):
    errors = []
    for i in range(0,12):
        x_train, x_test, y_train, y_test = make_data(month_data[i])

        ridge = calculate_ridge(x_train, x_test, y_train, y_test)
        lasso = calculate_lasso(x_train, x_test, y_train, y_test)
        decisiontree = calculate_decisiontree(x_train, x_test, y_train, y_test)
        kneighbors = calculate_kneighbors(x_train, x_test, y_train, y_test)
        linear= calculate_linear_reg(x_train, x_test, y_train, y_test)
        randomforest = calculate_randomforest(x_train, x_test, y_train, y_test)
        xgb =calculate_xgb(x_train, x_test, y_train, y_test)

        errors.append({f"ridge":ridge,"lasso":lasso,"decisiontree":decisiontree,
                "kneighbors":kneighbors,"linear":linear,"randomforest":randomforest,"xgb":xgb})
    return errors

In [125]:
errors = calculate_errors(data_months)

In [129]:
errors[1]



{'ridge': (0.011075696402284155, 0.006280715262375647),
 'lasso': (0.05992525947302542, 0.1056202492413188),
 'decisiontree': (0.0, 0.0017677754497907984),
 'kneighbors': (0.003673830459816446, 0.002481735322242497),
 'linear': (0.010661386349132855, 0.00623441048524434),
 'randomforest': (0.000889517482356323, 0.0011273348056097478),
 'xgb': (8.140274566150412e-07, 0.0016717620825358738)}

In [40]:
ridge_train =[]
ridge_test = []
lasso_train = []
lasso_test = []
for i in errors:
    ridge_train.append(i["ridge"][0])
    ridge_test.append(i["ridge"][1])
    lasso_train.append(i["lasso"][0])
    lasso_test.append(i["lasso"][1])

aylar = ["January","February","March","April","May","June","July","August","September","October","November","December"]

seri1  = pd.Series(ridge_train) 
seri2  = pd.Series(ridge_test) 
seri3  = pd.Series(lasso_train) 
seri4  = pd.Series(lasso_test) 
seri5 = pd.Series(aylar)

frame = {"months":seri5,"ridge_train":ridge_train, "ridge_test":ridge_test,"lasso_train":lasso_train,"lasso_test":lasso_test}
result = pd.DataFrame(frame)

In [41]:

data = result

fig = go.Figure()
fig.add_trace(go.Bar(x=data["months"],y=data["ridge_test"],name='Ridge Model Train Results', marker_color='indianred'))

fig.add_trace(go.Bar(x=data["months"], y=data["ridge_test"], name='Ridge Model Test Results',marker_color='lightsalmon'))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-45,title="(-1,1) normalization, 3 encoding")
fig.show()

In [42]:

fig = go.Figure()
fig.add_trace(go.Bar(x=data["months"],y=data["lasso_test"],name='Lasso Model Train Results', marker_color='indianred'))

fig.add_trace(go.Bar(x=data["months"], y=data["lasso_test"], name='Lasso Model Test Results',marker_color='lightsalmon'))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()